# 2-7-6. Neural Network 모델 (맛보기)

# 데이터 불러오기

In [32]:
import pandas as pd
import numpy as np
df = pd.read_csv('healthcare-dataset-3-dummy.csv')
df.shape

(4194, 16)

# 데이터셋 들여다 보기

In [20]:
df.head() # bmi 값에 결측값(빈 칸, NaN)이 있음에 주의.

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,ever_married_encoded,residence_type_encoded,gender_encoded_1.0,gender_encoded_2.0,work_type_encoded_0.0,work_type_encoded_2.0,work_type_encoded_3.0,smoking_status_encoded_0.0,smoking_status_encoded_1.0,smoking_status_encoded_3.0
0,67.0,0,1,228.69,36.6,1,1.0,1.0,1,0,0,1,0,0,1,0
1,61.0,0,0,202.21,NaN,1,1.0,0.0,0,0,0,0,1,0,0,0
2,80.0,0,1,105.92,32.5,1,1.0,0.0,1,0,0,1,0,0,0,0
3,49.0,0,0,171.23,34.4,1,1.0,1.0,0,0,0,1,0,0,0,1
4,79.0,1,0,174.12,24.0,1,1.0,0.0,0,0,0,0,1,0,0,0


# bmi 변수에 결측값이 몇 개나 있는지 확인

In [21]:
df['bmi'].value_counts(dropna=False) # NaN은 결측값(빈칸)을 의미함.

NaN     180
28.7     37
28.4     36
26.7     35
27.7     35
       ... 
49.9      1
56.1      1
48.7      1
66.8      1
14.1      1
Name: bmi, Length: 381, dtype: int64

# 타겟 변수 stroke의 값 분포 확인

In [33]:
df['stroke'].value_counts(dropna=False) # 1 값을 247개만 갖는 것에 주의.

0    3947
1     247
Name: stroke, dtype: int64

# 데이터셋을 타겟변수(stroke)와 나머지 입력변수로 구분

In [22]:
data = df.drop(['stroke'], axis=1)   # 타겟변수를 제외한 변수만 data에 저장.
target = df['stroke']                # 타겟변수만 target에 저장.

### 아래 구문은 이번 달 강의내 설명할 예정임.
### 지금은 강의의 효율적 진행을 위해 설명하지 않고 넘어갑니다.

from imblearn.under_sampling import RandomUnderSampler    # RandomUnderSampler를 import.
undersample = RandomUnderSampler(sampling_strategy=0.333, random_state=2) 
                                                          # 타겟변수의 소수 클래스 및 다수 클래스를
                                                          # 1:3의 비율(=1/3)로 언더샘플링.
data_under, target_under = undersample.fit_resample(data, target)
                                                          # data 및 target에 언더샘플링 적용.

# 데이터 50:50 분할

In [27]:
# 50:50 Data partition
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split( 
    data_under, target_under, test_size=0.5, random_state=42, stratify=target_under)

print("X_train shape:", X_train.shape) 
print("X_test shape:", X_test.shape) 

X_train shape: (494, 15)
X_test shape: (494, 15)


# 파이썬 라이브러리 Scikit-learn (sklearn) 결측값이 없어야 돌아감

# 때문에 입력 변수의 결측값(NaN)을 해당 변수값의 평균(mean)으로 대체

In [28]:
# interval 변수의 null value를 평균(mean)으로 impute. 
from sklearn.impute import SimpleImputer
imp= SimpleImputer(strategy = 'mean')  
X_train2= imp.fit_transform(X_train) 
X_test2= imp.fit_transform(X_test)   # X_train과 and X_test 둘 다 imputation 적용해야 함에 유의.

print("X_train2 shape:", X_train2.shape) 
print("X_test2 shape:", X_test2.shape) 

X_train2 shape: (494, 15)
X_test2 shape: (494, 15)


# Neural Network 모델 수립 및 실행

In [29]:
# Neural Network 모델 (Default 모델 with adam solver).
from sklearn.neural_network import MLPClassifier 
from sklearn.metrics import accuracy_score
clf_mlp = MLPClassifier(max_iter = 1000, random_state = 0)
clf_mlp.fit(X_train2, y_train)
pred = clf_mlp.predict(X_test2)  # 학습된 Classifier로 테스트 데이터셋 자료이용해서 타겟변수 예측값 생성.

# Neural Network 모델 결과 측정

In [30]:
print("Accuracy on test set:{:.5f}".format(accuracy_score(y_test, pred)))

Accuracy on test set:0.72065


# Neural Network 모델 끝! 이렇게 간단히? --> Yes!